In [7]:
import numpy as np

a = np.arange(2)
b = np.arange(8)

a = a.reshape((2,2,2))
b = b.reshape((2,2,2))



In [8]:
a, b

(array([[[0, 1],
         [2, 3]],
 
        [[4, 5],
         [6, 7]]]),
 array([[[0, 1],
         [2, 3]],
 
        [[4, 5],
         [6, 7]]]))

In [11]:
np.mean([a,b],axis=3)

array([[[0.5, 2.5],
        [4.5, 6.5]],

       [[0.5, 2.5],
        [4.5, 6.5]]])

In [ ]:
from config.read_configurations import config_PRMS_SNTEMP as config


In [14]:
from torch.nn.functional import softmax

def m_softmax(z):
    assert len(z.shape) == 2
    s = np.max(z, axis=1)
    s = s[:, np.newaxis] # necessary step to do broadcasting
    e_x = np.exp(z - s)
    div = np.sum(e_x, axis=1)
    div = div[:, np.newaxis] # dito
    return e_x / div

In [17]:
x1 = np.array([[1, 2, 3, 6]])
m_softmax(x1) #, m_softmax(x1)

array([[0.00626879, 0.01704033, 0.04632042, 0.93037047]])

In [19]:
x1 = np.array([[1, 2, 3, 6]])

m = m_softmax(x1)


In [20]:
n = []
for i in range(len(x1)):
    n.append(x1[i]*m[i])

In [23]:
n, np.sum(n)

([array([0.00626879, 0.03408066, 0.13896125, 5.58222279])], 5.761533493695909)

: 

-------
------
-----

In [1]:
import pandas as pd
import numpy as np
import os

In [ ]:
def getDataTs(self, args, varLst, doNorm=True, rmNan=True):
        if type(varLst) is str:
            varLst = [varLst]
        inputfile = os.path.join(os.path.realpath(args["forcing_path"]))
        inputfile_attr = os.path.join(os.path.realpath(args["attr_path"]))
        if inputfile.endswith(".csv"):
            dfMain = pd.read_csv(inputfile)
            dfMain_attr = pd.read_csv(inputfile_attr)
        elif inputfile.endswith(".feather"):
            dfMain = pd.read_feather(inputfile)
            dfMain_attr = pd.read_feather(inputfile_attr)
        else:
            print("data type is not supported")
            exit()
        sites = dfMain["site_no"].unique()
        tLst = tRange2Array(args["tRange"])
        tLstobs = tRange2Array(args["tRange"])
        # nt = len(tLst)
        ntobs = len(tLstobs)
        nNodes = len(sites)

        varLst_forcing = []
        varLst_attr = []
        for var in varLst:
            if var in dfMain.columns:
                varLst_forcing.append(var)
            elif var in dfMain_attr.columns:
                varLst_attr.append(var)
            else:
                print(var, "the var is not in forcing file nor in attr file")
        xt = dfMain.loc[:, varLst_forcing].values
        g = dfMain.reset_index(drop=True).groupby("site_no")
        xtg = [xt[i.values, :] for k, i in g.groups.items()]
        x = np.array(xtg)

        ## for attr
        if len(varLst_attr) > 0:
            x_attr_t = dfMain_attr.loc[:, varLst_attr].values
            x_attr_t = np.expand_dims(x_attr_t, axis=2)
            xattr = np.repeat(x_attr_t, x.shape[1], axis=2)
            xattr = np.transpose(xattr, (0, 2, 1))
            x = np.concatenate((x, xattr), axis=2)

        data = x
        C, ind1, ind2 = np.intersect1d(self.time, tLst, return_indices=True)
        data = data[:, ind2, :]
        # if os.path.isdir(out):
        #     pass
        # else:
        #     os.makedirs(out)
        # np.save(os.path.join(out, 'x.npy'), data)
        # if doNorm is True:
        #     data = transNorm(data, varLst, toNorm=True)
        # if rmNan is True:
        #     data[np.where(np.isnan(data))] = 0
        return np.swapaxes(data, 1, 0)



def loadData(args, trange):
    out_dict = dict()
    df = DataFrame_dataset(tRange=trange)
    # getting inputs for NN model:
    out_dict["x_NN"] = df.getDataTs(args, varLst=args["varT_NN"])
    out_dict["c_NN"] = df.getDataConst(args, varLst=args["varC_NN"])
    out_dict["obs"] = df.getDataTs(args, varLst=args["target"])
    if args["hydro_model_name"] != "None":
        out_dict["x_hydro_model"] = df.getDataTs(args, varLst=args["varT_hydro_model"])
        out_dict["c_hydro_model"] = df.getDataConst(args, varLst=args["varC_hydro_model"])
    if args["temp_model_name"] != "None":
        out_dict["x_temp_model"] = df.getDataTs(args, varLst=args["varT_temp_model"])
        out_dict["c_temp_model"] = df.getDataConst(args, varLst=args["varC_temp_model"])
    return out_dict

---
## Walking Through PRMS SAC-SMA testing in PGML here

---

In [5]:
from config.read_configurations import config_sacsma as args
import torch
import os
import numpy as np

from core.utils.randomseed_config import randomseed_config
from core.utils.master import loadModel, create_output_dirs
from MODELS.loss_functions.get_loss_function import get_lossFun
from MODELS.train_test import test_differentiable_model
from core.data_processing.data_loading import loadData


from core.utils import time


In [6]:
randomseed_config(seed=args["randomseed"][0])
# Creating output directories and adding it to args
args = create_output_dirs(args)

lossFun = get_lossFun(args)

modelFile = os.path.join(args["out_dir"], "model_Ep" + str(args["EPOCHS"]) + ".pt")
diff_model = torch.load(modelFile)


In [7]:
diff_model

diff_hydro_temp_model(
  (hydro_model): SACSMAMul()
  (NN_model): CudnnLstmModel(
    (linearIn): Linear(in_features=37, out_features=256, bias=True)
    (lstm): CudnnLstm()
    (linearOut): Linear(in_features=256, out_features=24, bias=True)
    (activation_sigmoid): Sigmoid()
  )
)

In [8]:
warm_up = args["warm_up"]
nmul = args["nmul"]
diff_model.eval()
# read data for test time range
dataset_dictionary = loadData(args, trange=args["t_test"])

In [9]:
dataset_dictionary['obs']

array([[[4.80e+01],
        [3.90e+01],
        [3.60e+01],
        ...,
        [4.10e+01],
        [5.20e-01],
        [1.14e+02]],

       [[4.60e+01],
        [3.90e+01],
        [3.60e+01],
        ...,
        [3.60e+01],
        [4.60e-01],
        [1.03e+02]],

       [[4.30e+01],
        [3.90e+01],
        [3.50e+01],
        ...,
        [3.40e+01],
        [4.40e-01],
        [9.80e+01]],

       ...,

       [[6.39e+02],
        [6.30e+01],
        [5.75e+02],
        ...,
        [3.50e+01],
        [9.00e-02],
        [1.24e+02]],

       [[7.52e+02],
        [7.20e+01],
        [5.99e+02],
        ...,
        [3.40e+01],
        [9.00e-02],
        [1.19e+02]],

       [[8.61e+02],
        [7.50e+01],
        [6.23e+02],
        ...,
        [3.40e+01],
        [8.00e-02],
        [1.15e+02]]])

dataset

In [10]:
from core.data_processing.normalization import transNorm

x_NN_scaled = transNorm(args, dataset_dictionary["x_NN"], varLst=args["varT_NN"], toNorm=True)
c_NN_scaled = transNorm(args, dataset_dictionary["c_NN"], varLst=args["varC_NN"], toNorm=True)
c_NN_scaled = np.repeat(np.expand_dims(c_NN_scaled, 0), x_NN_scaled.shape[0], axis=0)
dataset_dictionary["inputs_NN_scaled"] = np.concatenate((x_NN_scaled, c_NN_scaled), axis=2)
del x_NN_scaled, dataset_dictionary["x_NN"]

In [11]:
for key in dataset_dictionary.keys():
    dataset_dictionary[key] = torch.from_numpy(dataset_dictionary[key]).float()

args["batch_size"] = args["no_basins"]
nt, ngrid, nx = dataset_dictionary["inputs_NN_scaled"].shape
rho = args["rho"]

batch_size = args["batch_size"]
iS = np.arange(0, ngrid, batch_size)
iE = np.append(iS[1:], ngrid)
list_out_diff_model = []

In [12]:
from core.data_processing.model import (
    No_iter_nt_ngrid,
    take_sample_train,
    take_sample_test,
    converting_flow_from_ft3_per_sec_to_mm_per_day
)
from tqdm import tqdm


for i in tqdm(range(0, len(iS))):
    dataset_dictionary_sample = take_sample_test(args, dataset_dictionary, iS[i], iE[i])
    out_diff_model = diff_model(dataset_dictionary_sample)
    out_diff_model_cpu = {key: tensor.cpu
    ().detach() for key, tensor in out_diff_model.items()}
    # out_diff_model_cpu = tuple(outs.cpu().detach() for outs in out_diff_model)
    list_out_diff_model.append(out_diff_model_cpu)


  0%|          | 0/27 [00:00<?, ?it/s]d:\code_repos\hydro_ensemble\MODELS\NN_models\LSTM_models.py:99: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\RNN.cpp:982.)
  output, hy, cy, reserve, new_weight_buf = torch._cudnn_rnn(
  7%|▋         | 2/27 [00:28<05:55, 14.22s/it]


KeyboardInterrupt: 

In [16]:
dataset_dictionary.keys(), dataset_dictionary_sample.keys(), out_diff_model.keys()

(dict_keys(['c_NN', 'obs', 'x_hydro_model', 'c_hydro_model', 'inputs_NN_scaled']),
 dict_keys(['c_NN_sample', 'obs_sample', 'x_hydro_model_sample', 'c_hydro_model_sample', 'inputs_NN_scaled_sample']),
 dict_keys(['flow_sim', 'srflow', 'ssflow', 'gwflow', 'PET_hydro', 'AET_hydro', 'BFI_sim']))

In [17]:
len(dataset_dictionary['inputs_NN_scaled'])

5479

In [18]:
y_obs = converting_flow_from_ft3_per_sec_to_mm_per_day(args, dataset_dictionary["c_NN"],
                                                           dataset_dictionary["obs"][warm_up:, :, :])

In [23]:
y_obs

tensor([[[0.6701],
         [0.9128],
         [0.3254],
         ...,
         [0.1160],
         [0.0201],
         [0.2646]],

        [[0.6353],
         [0.8275],
         [0.3527],
         ...,
         [0.1160],
         [0.0213],
         [0.2646]],

        [[0.6245],
         [0.7507],
         [0.3620],
         ...,
         [0.1160],
         [0.0224],
         [0.2646]],

        ...,

        [[0.6940],
         [0.2687],
         [0.3827],
         ...,
         [0.1933],
         [0.0053],
         [0.4318]],

        [[0.8167],
         [0.3071],
         [0.3986],
         ...,
         [0.1877],
         [0.0053],
         [0.4144]],

        [[0.9351],
         [0.3199],
         [0.4146],
         ...,
         [0.1877],
         [0.0047],
         [0.4004]]])

In [ ]:
def test_differentiable_model(args, diff_model):
    warm_up = args["warm_up"]
    nmul = args["nmul"]
    diff_model.eval()
    # read data for test time range
    dataset_dictionary = loadData(args, trange=args["t_test"])
    np.save(os.path.join(args["out_dir"], "x.npy"), dataset_dictionary["x_NN"])  # saves with the overlap in the beginning
    # normalizing
    x_NN_scaled = transNorm(args, dataset_dictionary["x_NN"], varLst=args["varT_NN"], toNorm=True)
    c_NN_scaled = transNorm(args, dataset_dictionary["c_NN"], varLst=args["varC_NN"], toNorm=True)
    c_NN_scaled = np.repeat(np.expand_dims(c_NN_scaled, 0), x_NN_scaled.shape[0], axis=0)
    dataset_dictionary["inputs_NN_scaled"] = np.concatenate((x_NN_scaled, c_NN_scaled), axis=2)
    del x_NN_scaled, dataset_dictionary["x_NN"]
    # converting the numpy arrays to torch tensors:
    for key in dataset_dictionary.keys():
        dataset_dictionary[key] = torch.from_numpy(dataset_dictionary[key]).float()

    # args_mod = args.copy()
    args["batch_size"] = args["no_basins"]
    nt, ngrid, nx = dataset_dictionary["inputs_NN_scaled"].shape
    rho = args["rho"]

    # Making lists of the start and end indices of the basins for each batch.
    batch_size = args["batch_size"]
    iS = np.arange(0, ngrid, batch_size)    # Start index list.
    iE = np.append(iS[1:], ngrid)   # End.
    
    list_out_diff_model = []
    for i in tqdm(range(0, len(iS)), unit='Batch'):
        dataset_dictionary_sample = take_sample_test(args, dataset_dictionary, iS[i], iE[i])

        out_diff_model = diff_model(dataset_dictionary_sample)
        # Convert all tensors in the dictionary to CPU
        out_diff_model_cpu = {key: tensor.cpu().detach() for key, tensor in out_diff_model.items()}
        # out_diff_model_cpu = tuple(outs.cpu().detach() for outs in out_diff_model)
        list_out_diff_model.append(out_diff_model_cpu)

    # getting rid of warm-up period in observation dataset and making the dimension similar to
    # converting numpy to tensor
    # y_obs = torch.tensor(np.swapaxes(y_obs[:, warm_up:, :], 0, 1), dtype=torch.float32)
    # c_hydro_model = torch.tensor(c_hydro_model, dtype=torch.float32)
    y_obs = converting_flow_from_ft3_per_sec_to_mm_per_day(args, dataset_dictionary["c_NN"],
                                                           dataset_dictionary["obs"][warm_up:, :, :])

    return list_out_diff_model, y_obs

In [29]:
len(dataset_dictionary['x_NN'])

5479

---
## Walking Through HBV testing in dPLHBV here

---

In [3]:
import random
import numpy as np

## fix the random seeds
import torch
randomseed = 111111
random.seed(randomseed)
torch.manual_seed(randomseed)
np.random.seed(randomseed)
torch.cuda.manual_seed(randomseed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
from hydroDL import master, utils


## GPU setting
testgpuid = 0
torch.cuda.set_device(testgpuid)

## setting options, keep the same as your training
PUOpt = 0  # 0 for All; 1 for PUB; 2 for PUR;
buffOptOri = 0  # original buffOpt, must be same as what you set for training
buffOpt = 0  # control load training data 0: do nothing; 1: repeat first year; 2: load one more year
forType = 'daymet'

## Hyperparameters, keep the same as your training setup
BATCH_SIZE = 100
RHO = 365
HIDDENSIZE = 256
Ttrain = [19801001, 19951001]  # Training period
# Ttrain = [19891001, 19991001]  # PUB/PUR period
Tinv = [19801001, 19951001] # dPL Inversion period
# Tinv = [19891001, 19991001]  # PUB/PUR period
Nfea = 12 # number of HBV parameters
BUFFTIME = 365
routing = True
Nmul = 16
comprout = False
compwts = False
pcorr = None

Ttest = [19951001, 20101001]  # testing period
TtestLst = utils.time.tRange2Array(Ttest)
TtestLoad = [19951001, 20101001]  

testbatch = 50  # forward number of "testbatch" basins each time to save GPU memory. You can set this even smaller to save more.
testepoch = 50

testseed = 111111


loading package hydroDL


In [36]:
# Define root directory of database and saved output dir
# Modify this based on your own location of CAMELS dataset and saved models
rootDatabase = os.path.join(os.path.sep, 'D:\data', 'Camels')  # CAMELS dataset root directory
camels.initcamels(rootDatabase)  # initialize three camels module-scope variables in camels.py: dirDB, gageDict, statDict

NameError: name 'camels' is not defined

## From testMulti.ipynb

In [ ]:
# # Instantiate the model
# lstm = hydroEnsemble(num_models=3, hidden_size=256, num_layers=1)


# # Create a sample input tensor
# batch_size = 1
# sequence_length = 671
# num_models = 3
# input_tensor = torch.randn(sequence_length, num_models)
# input_array = np.random.rand(sequence_length, num_models)

# # Forward pass through the model
# preds= lstm(torch.tensor(input_tensor, dtype=torch.float))

In [ ]:
for key in model_output['marrmot_PRMS'][0].keys():
    if len(model_output['marrmot_PRMS'][0][key].shape) == 3:
        dim = 1
    else:
        dim = 0
    concatenated_tensor = torch.cat([d[key] for d in model_output], dim=dim)
    file_name = key + ".npy"
    np.save(os.path.join(SAVE_PATH, args["testing_dir"], file_name), concatenated_tensor.numpy())

In [ ]:
array1 = np.array([[1, 2],
                   [3, 4],
                   [5, 6]])

# Create a 1x3 array
a2 = np.array([7, 8, 9])

a = np.stack((a2, a2*2), axis=1)
# print(a)

b = np.hstack((a, a2.reshape(-1,1)))


b, np.amax(b, axis=1)




# def calc_nse(pred, target):
#     """
#     Currently returns the overall nse per basin.

#     Note: modify this to allow per day per basin as well.
#     """
#     # ngrid: number of basins
#     # nt: number of timesteps (in days usually)
#     ngrid, nt = pred.shape
#     NSE = np.full(ngrid, np.nan)

#     print(len(pred[670,:]), len(pred))
#     for k in range(0, ngrid):
#         x = pred[k, :]
#         y = target[k, :]
#         ind = np.where(np.logical_and(~np.isnan(x), ~np.isnan(y)))[0]
#         if ind.shape[0] > 0:
#             xx = x[ind]
#             yy = y[ind]

#             if ind.shape[0] > 1:
#                 yymean = yy.mean()

#                 SST = np.sum((yy-yymean)**2)
#                 SSRes = np.sum((yy-xx)**2)
#                 NSE[k] = 1-SSRes/SST

#     return NSE



# for i, (x,y) in enumerate(zip(preds, obs)):
#     # print(i)
#     # print(x.shape)
#     nse = calc_nse(np.swapaxes(x.squeeze(), 1, 0), np.swapaxes(y.squeeze(), 1, 0))

In [ ]:
# OLD HYDRO MULTI MOSAIC MODEL

class hydroEnsemble(torch.nn.Module):
    # Wrapper for multiple hydrologic models.
    # In future, consider just passing the models you want to ensemble explicitly.
    def __init__(self, num_models, hidden_size, num_layers):
        super(hydroEnsemble, self).__init__()

        self.lstm = torch.nn.LSTM(num_models, hidden_size, num_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, num_models)  # Two models (modelA and modelB)

        # self.modelA = modelA
        # self.modelB = modelB
        # self.classifier = torch.nn.Linear(4, 2)

    def forward(self, x):
        # x is the input sequence tensor with shape (batch_size, sequence_length, num_models)

        # Setting randomseed for deterministic output.
        randomseed_config(0)

        # Add batch dimension to input and convert to tensor.
        x_exp = x.unsqueeze(0)

        # LSTM layer
        lstm_out, _ = self.lstm(x_exp)

        # Fully connected layer
        fc_out = self.fc(lstm_out)

        # Apply softmax activation to obtain weights
        weights = torch.nn.functional.softmax(fc_out, dim=2).squeeze()

        # Weighted combination of predictions.
        weighted_preds = np.multiply(weights.detach(), x)

        # Or take the max weight and return the corresponding value.
        max_vals, _ = torch.max(weights, dim=1)
        btensor = torch.zeros_like(weights)
        btensor[weights==max_vals.view(-1,1)] = 1
        weighted_preds = np.multiply(btensor.detach(), x)

        preds = torch.sum(weighted_preds, dim=1)

        # All tensors
        # return preds, weights, weighted_preds
        return preds


In [ ]:



class PGMLHydroModel(torch.nn.Module):
    """
    Differentiable hydro model code from F. Rahmani PGML_STemp_with_Snow.
    Use for PRMS, SAC-SMA, and unmodified HBV.
    """
    def __init__(self, args):
        super(diff_hydro_temp_model, self).__init__()
        self.args = args
        self.get_model()

    def get_NN_model_dim(self) -> None:
        self.nx = len(self.args["varT_NN"] + self.args["varC_NN"])

        # output size of NN
        if self.args["hydro_model_name"] != "None":
            if self.args["routing_hydro_model"] == True:  # needs a and b for routing with conv method
                self.ny_hydro = self.args["nmul"] * (len(self.hydro_model.parameters_bound)) + len(
                    self.hydro_model.conv_routing_hydro_model_bound)
            else:
                self.ny_hydro = self.args["nmul"] * len(self.hydro_model.parameters_bound)
        else:
            self.ny_hydro = 0

        # SNTEMP  # needs a and b for calculating different source flow temperatures with conv method
        if self.args["temp_model_name"] != "None":
            if self.args["routing_temp_model"] == True:
                self.ny_temp = self.args["nmul"] * (len(self.temp_model.parameters_bound)) + len(
                    self.temp_model.conv_temp_model_bound)
            else:
                self.ny_temp = self.args["nmul"] * len(self.temp_model.parameters_bound)
            if self.args["lat_temp_adj"] == True:
                self.ny_temp = self.ny_temp + self.args["nmul"]
        else:
            self.ny_temp = 0
        # if self.args["hydro_model_name"] == "HBV":   # no need to have a PET to AET coef
        #     self.ny_PET = 0
        # elif self.args["hydro_model_name"] == "marrmot_PRMS":   # need a PET to AET coef
        #     self.ny_PET = self.args["nmul"]
        # if self.args["potet_module"] in ["potet_hargreaves", "potet_hamon", "dataset"]:
        #     self.ny_PET = self.args["nmul"]
        self.ny = self.ny_hydro + self.ny_temp # + self.ny_PET

    def get_model(self) -> None:
        # hydro_model_initialization
        if self.args["hydro_model_name"] != "None":
            if self.args["hydro_model_name"] == "marrmot_PRMS":
                self.hydro_model = prms_marrmot()
            elif self.args["hydro_model_name"] == "marrmot_PRMS_gw0":
                self.hydro_model = prms_marrmot_gw0()
            elif self.args["hydro_model_name"] == "HBV":
                self.hydro_model = HBVMul()
            elif self.args["hydro_model_name"] == "SACSMA":
                self.hydro_model = SACSMAMul()
            elif self.args["hydro_model_name"] == "SACSMA_with_snow":
                self.hydro_model = SACSMA_snow_Mul()
            elif self.args["hydro_model_name"] != "None":
                print("hydrology (streamflow) model type has not been defined")
                exit()
            # temp_model_initialization
        if self.args["temp_model_name"] != "None":
            if self.args["temp_model_name"] == "SNTEMP":
                self.temp_model = SNTEMP_flowSim()  # this model needs a hydrology model as backbone
            elif self.args["temp_model_name"] == "SNTEMP_gw0":
                self.temp_model = SNTEMP_flowSim_gw0()  # this model needs a hydrology model as backbone, and 4 outflow
            elif self.args["temp_model_name"] != "None":
                print("temp model type has not been defined")
                exit()
        # get the dimensions of NN model based on hydro modela and temp model
        self.get_NN_model_dim()
        # NN_model_initialization
        if self.args["NN_model_name"] == "LSTM":
            self.NN_model = CudnnLstmModel(nx=self.nx,
                                           ny=self.ny,
                                           hiddenSize=self.args["hidden_size"],
                                           dr=self.args["dropout"])
        elif self.args["NN_model_name"] == "MLP":
            self.NN_model = MLPmul(self.args, nx=self.nx, ny=self.ny)
        else:
            print("NN model type has not been defined")
            exit()

    def breakdown_params(self, params_all):
        params_dict = dict()
        params_hydro_model = params_all[-1, :, :self.ny_hydro]
        params_temp_model = params_all[-1, :, self.ny_hydro: (self.ny_hydro + self.ny_temp)]
        # if self.ny_PET > 0:
        #     params_dict["params_PET_model"] = torch.sigmoid(params_all[-1, :, (self.ny_hydro + self.ny_temp):])
        # else:
        #     params_dict["params_PET_model"] = None


        # Todo: I should separate PET model output from hydro_model and temp_model.
        #  For now, evap is calculated in both models individually (with same method)

        if self.args['hydro_model_name'] != "None":
            # hydro params
            params_dict["hydro_params_raw"] = torch.sigmoid(
                params_hydro_model[:, :len(self.hydro_model.parameters_bound) * self.args["nmul"]]).view(
                params_hydro_model.shape[0], len(self.hydro_model.parameters_bound),
                self.args["nmul"])
            # routing params
            if self.args["routing_hydro_model"] == True:
                params_dict["conv_params_hydro"] = torch.sigmoid(
                    params_hydro_model[:, len(self.hydro_model.parameters_bound) * self.args["nmul"]:])
            else:
                params_dict["conv_params_hydro"] = None

        if self.args['temp_model_name'] != "None":
            # hydro params
            params_dict["temp_params_raw"] = torch.sigmoid(
                params_temp_model[:, :len(self.temp_model.parameters_bound) * self.args["nmul"]]).view(
                params_temp_model.shape[0], len(self.temp_model.parameters_bound),
                self.args["nmul"])
            # convolution parameters for ss and gw temp calculation
            if self.args["routing_temp_model"] == True:
                params_dict["conv_params_temp"] = torch.sigmoid(params_temp_model[:, -len(self.temp_model.conv_temp_model_bound):])
            else:
                print("it has not been defined yet what approach should be taken in place of conv")
                exit()
        return params_dict


    def forward(self, dataset_dictionary_sample):
        params_all = self.NN_model(dataset_dictionary_sample["inputs_NN_scaled_sample"][self.args["warm_up"]:, :, :])
        # breaking down the parameters to different pieces for different models (PET, hydro, temp)
        params_dict = self.breakdown_params(params_all)
        if self.args['hydro_model_name'] != "None":
            # hydro model
            flow_out = self.hydro_model(
                dataset_dictionary_sample["x_hydro_model_sample"],
                dataset_dictionary_sample["c_hydro_model_sample"],
                params_dict['hydro_params_raw'],
                self.args,
                # PET_param=params_dict["params_PET_model"],  # PET is in both temp and flow model
                warm_up=self.args["warm_up"],
                routing=self.args["routing_hydro_model"],
                conv_params_hydro=params_dict["conv_params_hydro"]
            )
            # baseflow index percentage
            flow_out["BFI_sim"] = 100 * (torch.sum(flow_out["gwflow"], dim=0) / (
                    torch.sum(flow_out["flow_sim"], dim=0) + 0.00001))[:, 0]

            if self.args['temp_model_name'] != "None":
                # source flow calculation and converting mm/day to m3/ day
                source_flows_dict = source_flow_calculation(self.args, flow_out,
                                                              dataset_dictionary_sample[
                                                                  "c_NN_sample"],
                                                              after_routing=True)
                # temperature model
                temp_out = self.temp_model.forward(dataset_dictionary_sample["x_temp_model_sample"],
                                                   dataset_dictionary_sample["c_temp_model_sample"],
                                                   params_dict["temp_params_raw"],
                                                   conv_params_temp=params_dict["conv_params_temp"],
                                                   args=self.args,
                                                   PET=flow_out["PET_hydro"] * (1 / (1000 * 86400)),   # converting mm/day to m/sec,
                                                   source_flows=source_flows_dict)

                return {**flow_out, **temp_out}   # combining both dictionaries
            else:
                return flow_out



In [ ]:

class HydroMultimodel(nn.Module):
    """
    A wrapper for managing a collection of (trained) hydromodels, 
    and applyinf different ensembling methods.
    """
    def __init__(self, modelList, argList):
        super(HydroMultimodel, self).__init__()
        """
        Instantiate hydro models to be ensembled.
        """

        self.ensemble_mtd = None  # Ensemble method 
        # self.defaultKeys = {}
        # self.defaultKeys['models'] = ['HBV', 'dPLHBV', 'dPLHBV_dp', 'SACSMA', 'SACSMA_snow', 'marrmot_PRMS']
        # self.defaultKeys['ensemble_mtds'] = {}  
          
        self.modelDict = m.createDictFromKeys(modelList)
        self.argDict = m.createDictFromKeys(modelList, mtd='ref', dat=argList)
        self.initModel()


    def initModel(self, mode='all', *args, **kwargs):
        """
        Instantiate hydro models to be ensembled.
        """
        if mode == 'all':
            for mod in self.modelsDict:
                if mod in ['HBV', 'SACSMA', 'SACSMA_snow', 'marrmot_PRMS']:
                    self.modelDict[mod] = PGMLHydroModel(self.argDict[mod])
                elif mod in ['dPLHBV']:
                    self.modelDict[mod] = rnn.MultiInv_HBVModel(*args, **kwargs)
                elif mod in ['dPLHBV_dp']:
                    self.modelDict[mod] = rnn.MultiInv_HBVTDModel(*args, **kwargs)
                else:
                    raise ValueError("Invalid hydrology model specified.")     
                 
        elif mode in self.modelDict:
            if mod in ['HBV', 'SACSMA', 'SACSMA_snow', 'marrmot_PRMS']:
                self.modelDict[mod] = PGMLHydroModel(self.argDict[mod])
            elif mod in ['dPLHBV_dp']:
                self.modelDict[mod] = rnn.MultiInv_HBVModel(*args, **kwargs)
            else:
                self.modelDict[mod] = rnn.MultiInv_HBVTDModel(*args, **kwargs) 
        else:
            raise ValueError("Invalid hydrology model specified.")
        

    def multimodel_ensemble():
        mm_ensemble = F.MultiModelEnsemble()


    def fuse_on_avg():
        # fuse with average of streamflows here.
        x = 1


    def fuse_on_md():
        # Fuse with median of streamflows here.
        x = 1


    def forward(self, *args, **kwargs):
        for key in self.modelsDict:
            self.modelDict[key](*args, **kwargs)
      
